# Ejercicio 4 de laboratorio

In [2]:
from ucimlrepo import fetch_ucirepo
from random import randint
from random import shuffle
import pandas as pd

In [3]:
adult = fetch_ucirepo(id=2) 

In [4]:
iris = fetch_ucirepo(id=53) 

In [5]:
X = iris.data.features 
Y = iris.data.targets 

In [6]:
union = pd.concat([X,Y],axis=1)
union2 = pd.concat([adult.data.features,adult.data.targets],axis=1)
union.iloc[0]

sepal length            5.1
sepal width             3.5
petal length            1.4
petal width             0.2
class           Iris-setosa
Name: 0, dtype: object

## Hold out

El hold out specifica que porcentaje del dataset se va a dejar afuera para validación/testing. en este caso un r, donde r pertenece a [0,1]

In [7]:
def holdOut(li, r): # Recibe un listado iterable que contenga los filas o instancias
    # Logra separar en classes discretas
    l = li.drop_duplicates(ignore_index=True)
    tam = len(l.index)
    elems = list(range(tam))
    clases = {}
    for a in range(tam):
        # print(l.iat[a, -1])  # Se asume que la clase esta en la ultima columna
        c = l.iat[a,-1]
        if c not in clases:
            clases[c] = []
        else:
            clases[c].append(a)
    ent = []
    val = []
    for k in clases:
        t = len(clases[k])
        v = int(t*r)
        shuffle(clases[k])
        val += clases[k][0:v]
        ent += clases[k][v:]
    return l.iloc[ent],l.iloc[val]

ent,val = holdOut(union, 0.3)

In [8]:
ent2,val2 = holdOut(union2,0.3)

In [9]:
def elemComun(a, b): # a y b son datasets de pandas, retorna true si existen elementos en comun
    tamA = len(a.index)
    tamB = len(b.index)
    df3 = pd.concat([a,b], ignore_index=True)
    df3 = df3.drop_duplicates(ignore_index=True)
    tamRes = len(df3.index)
    print(tamA,tamB,tamRes)
    if((tamA + tamB) > tamRes):
        return True # Existen elementos en comun
    else:
        return False # No existen elementos en comun
    

In [10]:
print(elemComun(ent,val)) # Observamos si la separacion causo elementos repetidos
print(elemComun(ent2,val2))
print(elemComun(ent,ent))

102 42 144
False
34168 14641 48809
False
102 102 102
True


In [18]:
print(ent)
print(val)

     sepal length  sepal width  petal length  petal width           class
45            4.6          3.2           1.4          0.2     Iris-setosa
39            4.5          2.3           1.3          0.3     Iris-setosa
34            5.0          3.2           1.2          0.2     Iris-setosa
38            5.0          3.5           1.3          0.3     Iris-setosa
10            5.4          3.7           1.5          0.2     Iris-setosa
..            ...          ...           ...          ...             ...
104           4.9          2.5           4.5          1.7  Iris-virginica
109           6.4          2.7           5.3          1.9  Iris-virginica
126           6.4          2.8           5.6          2.1  Iris-virginica
112           5.8          2.8           5.1          2.4  Iris-virginica
140           6.8          3.2           5.9          2.3  Iris-virginica

[102 rows x 5 columns]
     sepal length  sepal width  petal length  petal width            class
15           

## K-fold cross-validation

No estoy seguro de como recibir una función de predicción, entonces va a retornar k tuples, donde el primer conjunto es el entrenamiento, y el segundo los resultados

In [11]:
def kfold(f, k):
    frame = f.drop_duplicates(ignore_index=True)
    nums = list(range(len(frame.index)))
    shuffle(nums)
    # El ultimo conjunto tendra mas elementos de ser el caso de sobra
    part = []
    div = int((len(frame.index))/k)
    # Particionamos el frame
    for a in range(k-1):
        part.append(frame.iloc[nums[a*div:(a+1)*div]])
    part.append(frame.iloc[nums[(k-1)*div:]])
    # Part tiene las particiones en k
    # Ahora realizamos los tuples
    ret =[]
    for a in range(k):
        ret.append( (pd.concat(part[:a] + part[a+1:]),part[a]) )
    return ret

In [12]:
res = kfold(union, 3)
for a in res:
    #print("--")
    #print(a[0])
    #print(a[1])
    #print("---")
    print(len(a[0].index), len(a[1].index))
    print(elemComun(a[0], a[1]))
res2 = kfold(union2, 10)
for a in res2:
    print(elemComun(a[0], a[1]))

98 49
98 49 147
False
98 49
98 49 147
False
98 49
98 49 147
False
43932 4881 48813
False
43932 4881 48813
False
43932 4881 48813
False
43932 4881 48813
False
43932 4881 48813
False
43932 4881 48813
False
43932 4881 48813
False
43932 4881 48813
False
43932 4881 48813
False
43929 4884 48813
False


In [13]:
def leaveOneOut(f):
    res = []
    frame = f.drop_duplicates(ignore_index=True)
    tam = len(frame.index)
    for a in range(tam):
        r = list(range(tam))
        del r[a]
        res.append( (frame.iloc[r], frame.iloc[a]) )
    return res

In [14]:
res = leaveOneOut(union)

In [17]:
print(len(union2.index)) # Debido al tamanio del dataset de adulto, necesitamos en memoria (607kB)×49k=29.743 GB. (intente y me crasheo la maquina)
# Probablemente se necesitaria realizar una version que reciba de parametro el modelo entrenado.

48842


In [15]:
for a in res:
    print(a[0])
    print(a[1])
    print("---")

     sepal length  sepal width  petal length  petal width           class
1             4.9          3.0           1.4          0.2     Iris-setosa
2             4.7          3.2           1.3          0.2     Iris-setosa
3             4.6          3.1           1.5          0.2     Iris-setosa
4             5.0          3.6           1.4          0.2     Iris-setosa
5             5.4          3.9           1.7          0.4     Iris-setosa
..            ...          ...           ...          ...             ...
142           6.7          3.0           5.2          2.3  Iris-virginica
143           6.3          2.5           5.0          1.9  Iris-virginica
144           6.5          3.0           5.2          2.0  Iris-virginica
145           6.2          3.4           5.4          2.3  Iris-virginica
146           5.9          3.0           5.1          1.8  Iris-virginica

[146 rows x 5 columns]
sepal length            5.1
sepal width             3.5
petal length            1.4
peta